# Options_Trading_5

In [1]:
from option import Option

For this assignment you can use the functions in the black_scholes.py module, which should be included with this assignment.

In [2]:
import black_scholes as bs

### Dummy data

In [3]:
call = Option('call', 1, 100)
put = Option('put', 0.5, 110)

option_positions = [(call, 100), (put, -50), (call, 50), (put, -100), (call, -50), (put, +200) ]

S = 100  # Underlying price
r = 0  # Interest rate
sigma = 0.2  # Volatility

from scipy import stats
import numpy as np

_norm_cdf = stats.norm(0, 1).cdf
_norm_pdf = stats.norm(0, 1).pdf


def _d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


def _d2(S, K, T, r, sigma):
    return _d1(S, K, T, r, sigma) - sigma * np.sqrt(T)


def call_value(S, K, T, r, sigma):
    return S * _norm_cdf(_d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * _norm_cdf(_d2(S, K, T, r, sigma))

def put_value(S, K, T, r, sigma):
    return np.exp(-r * T) * K * _norm_cdf(-_d2(S, K, T, r, sigma)) - S * _norm_cdf(-_d1(S, K, T, r, sigma))

def call_delta(S, K, T, r, sigma):
    return _norm_cdf(_d1(S, K, T, r, sigma))

def put_delta(S, K, T, r, sigma):
    return call_delta(S, K, T, r, sigma) - 1

def call_vega(S, K, T, r, sigma):
    return S * _norm_pdf(_d1(S, K, T, r, sigma)) * np.sqrt(T)

def put_vega(S, K, T, r, sigma):
    return call_vega(S, K, T, r, sigma)

In [4]:
option_positions = [(call, 100), (put, -50), (call, 50), (put, -100), (call, -50), (put, +200) ]
list2 = [x[1] for x in option_positions if x[0] == call]
list2

[100, 50, -50]

In [5]:
call = Option('call', 1, 100)
put = Option('put', 0.5, 110)

call.tte
call.strike

100

### Option delta

Write a function that calculates the black-scholes option delta given an option and market data.

input: Option, S, r, sigma

output: option delta

In [6]:
def option_delta(option, S, r, sigma):
    
    if option.kind == 'call':
        K = option.strike
        T = option.tte
        c_delta = call_delta(S, K, T, r, sigma)
        return c_delta
    
    if option.kind == 'put':
        K = option.strike
        T = option.tte
        p_delta = put_delta(S, K, T, r, sigma)
        return p_delta

option_delta(put, S, r, sigma)

-0.7268235059774313

### Total Delta Position

Write a function that calculates the total delta exposure of your position.

Your function is given a list of options with a corresponding position.
Calculate the delta of the total position.


input: option_positions (list of (Option, lots)), S, r, Sigma

output: Total delta position

In [7]:
def total_delta(option_positions, S, r, sigma):  
    
    total_position_calls = 0 
    total_position_puts = 0
    
    for kind in option_positions:
        if kind[0].kind == 'call':
            K = kind[0].strike
            T = kind[0].tte
            c_delta = call_delta(S, K, T, r, sigma)
            total_position_calls += kind[1]*c_delta
        
        if kind[0].kind == 'put':
            K = kind[0].strike
            T = kind[0].tte
            p_delta = put_delta(S, K, T, r, sigma)
            total_position_puts += kind[1]*p_delta
            
    return total_position_calls + total_position_puts

total_delta(option_positions, S, r, sigma)

17.641608428831347

### Delta hedge options

Write a function that calculates the number of stocks you need to buy/sell to delta hedge your option positions.

Remember: you can only trade a whole number of stocks. For this exercise always round the fractional number of stocks you would ideally buy/sell towards zero.

input: option_positions (list of (Option, lots)), S, r, Sigma

output: Number of stocks to buy (positive) or sell (negative)

In [9]:
def hedge(option_positions, S, r, sigma):
    #buy call = sell stock => positive delta
    #buy put = buy stock => negative delta 
    delta = total_delta(option_positions, S, r, sigma)

    total_position_calls = 0 
    total_position_puts = 0
    
    for kind in option_positions:
        if kind[0].kind == 'call':
            K = kind[0].strike
            T = kind[0].tte
            c_delta = call_delta(S, K, T, r, sigma)
            total_position_calls += kind[1]*c_delta
            
        
        if kind[0].kind == 'put':
            K = kind[0].strike
            T = kind[0].tte
            p_delta = put_delta(S, K, T, r, sigma)
            total_position_puts += kind[1]*p_delta
            
    return round((total_position_calls + total_position_puts) * -1)       
hedge(option_positions, S, r, sigma)            

-18.0

### Option gamma

Write a function that calculates the black-scholes option gamma given an option and market data.

input: Option, S, r, sigma

output: option gamma

In [57]:
import math 
def option_gamma(option, S, r, sigma):
    
    if option.kind == 'call':
        K = option.strike
        T = option.tte
        d1 = _d1(S, K, T, r, sigma)
        #print(d1)
        d1_numerator = math.exp(-(d1**2)/2)
        #print(d1_numerator)
        d1_denominator = np.sqrt((2*math.pi))
        __d1 = d1_numerator/d1_denominator
        result = __d1 / (S * (sigma * np.sqrt(T)))
        return result 
    if option.kind == 'put':
        K = option.strike
        T = option.tte
        d1 = _d1(S, K, T, r, sigma)
        d1_numerator = math.exp(-(d1**2)/2)
        d1_denominator = np.sqrt((2*math.pi))
        __dd = d1_numerator/d1_denominator
        result = __dd / (S * (sigma * np.sqrt(T)))
        return result
    
option_gamma(call, S, r, sigma)

0.01984762737385059

### Hedge (Gamma-Delta) exposures

Write a function that determines the position you need to take in option_a and option_b to hedge the given delta and gamma exposure.
For this exercise you are allowed to trade a fractional number of lots.

Example:
    option_a: delta: 0.2, gamma: 0.4
    option_b: delta: 0.3, gamma: 0.9
    You have a current delta exposure of -0.6 and a gamma exposure of -1.5.
    You can hedge this exposure by buying 1 lot of option_b and buying 3/2 lots of option_a.
    Your function should return (3/2, 1)


input: Option, Option, delta, gamma, S, r, sigma

output: Lots of a, lots of b (positive if buying, negative if selling)

In [67]:
def hedge_exposures(option_a, option_b, delta, gamma, S, r, sigma):
    
    delta_a = option_delta(option_a,S,r,sigma)
    gamma_a = option_gamma(option_a,S,r,sigma)
    
    delta_b = option_delta(option_b,S,r,sigma)
    gamma_b = option_gamma(option_b,S,r,sigma)
    
    beta = (delta_a*gamma - gamma_a*delta)/(delta_a*gamma_b-gamma_a*delta_b)
    alpha = (delta-beta*delta_b)/delta_a
    
    return -alpha, -beta

hedge_exposures(put, call, -3, -1.5, S, r, sigma)

(27.6614746063411, 42.80070118360622)